# Save and Load Checkpoints

In this tutorial, we will explore how to save and load checkpoints in `brainstate` by using the `orbax` library and `braintools` library which provide a more lightweight approach. This is particularly useful for saving the state of your model during training so that you can resume training from where you left off or use the trained model for inference later. The following example demonstrates how to use `orbax` and `braintools`'s checkpointing functionality with a simple MLP model.

First you can install the `orbax` library by running the following command:

`pip install orbax-checkpoint`

You may also install directly from GitHub, using the following command. This can be used to obtain the most recent version of Orbax.

`pip install 'git+https://github.com/google/orbax/#subdirectory=checkpoint'`

You can install the `braintools` library by running the following command:

`pip install braintools`

First, let's import the necessary libraries.

In [1]:
import tempfile
import os

import jax
import jax.numpy as jnp
import orbax.checkpoint as orbax
import braintools
import brainstate 

## Define the Model
We define a simple Multi-Layer Perceptron (MLP) model using `brainstate`.

In [2]:
class MLP(brainstate.nn.Module):
    def __init__(self, din: int, dmid: int, dout: int):
        super().__init__()
        self.dense1 = brainstate.nn.Linear(din, dmid)
        self.dense2 = brainstate.nn.Linear(dmid, dout)

    def __call__(self, x: jax.Array) -> jax.Array:
        x = self.dense1(x)
        x = jax.nn.relu(x)
        x = self.dense2(x)
        return x

## Create the Model
We create an instance of the model with a given seed for reproducibility.

In [3]:
SEED = 42
brainstate.random.seed(SEED)   # set seed in brainstate
model1 = MLP(10, 20, 30)    # create model
model1

MLP(
  dense1=Linear(
    in_size=(10,),
    out_size=(20,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[20]),
        'weight': ShapedArray(float32[10,20])
      }
    )
  ),
  dense2=Linear(
    in_size=(20,),
    out_size=(30,),
    w_mask=None,
    weight=ParamState(
      value={
        'bias': ShapedArray(float32[30]),
        'weight': ShapedArray(float32[20,30])
      }
    )
  )
)

## Save the Model State

### Save the Model State Using `orbax`
We save the model's parameters to a checkpoint file.

In [4]:
tmpdir = tempfile.mkdtemp()    # create temporary directory
state_tree = brainstate.graph.treefy_states(model1)    # convert model to state tree
checkpointer = orbax.PyTreeCheckpointer()   # create checkpointer
checkpointer.save(os.path.join(tmpdir, 'state'), state_tree)    # save state tree

Now, we've saved the model's parameters to the checkpoint files in `tmpdir/state` by using the `orbax` library.

### Save the Model State Using `braintools`

In [5]:
checkpoint = brainstate.graph.states(model1).to_nest()   # convert model to nest
braintools.file.msgpack_save(os.path.join(tmpdir, 'state.msgpack'), checkpoint)    # save checkpoint

Saving checkpoint into C:\Users\adadu\AppData\Local\Temp\tmplky_atoo\state.msgpack


Now, we've saved the model's parameters to the checkpoint files in `tmpdir/state.msgpack` by using the `braintools` library.

## Load the Model State

### Load the Model State Using `orbax`
Let's load the model's parameters from the checkpoint files.

In [6]:
# create that model with abstract shapes
brainstate.random.seed(0)
model2 = MLP(10, 20, 30)
state_tree = brainstate.graph.treefy_states(model2)

# Load the parameters from checkpoint files
checkpointer = orbax.PyTreeCheckpointer()
state_tree = checkpointer.restore(os.path.join(tmpdir, 'state'), item=state_tree)

# update the model with the loaded state
brainstate.graph.update_states(model2, state_tree)

C:\Users\adadu\miniconda3\envs\bdp\Lib\site-packages\orbax\checkpoint\_src\serialization\type_handlers.py:1269: UserWarning: Sharding info not provided when restoring. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


### Load the Model State Using `braintools`
Let's load the model's parameters from the checkpoint files.

In [7]:
# 创建一个有着相同结构的模型
brainstate.random.seed(0)
model3 = MLP(10, 20, 30)
checkpoint = brainstate.graph.states(model3).to_nest()

# 从msgpack文件读取模型参数
braintools.file.msgpack_load(os.path.join(tmpdir, 'state.msgpack'), checkpoint)

Loading checkpoint from C:\Users\adadu\AppData\Local\Temp\tmplky_atoo\state.msgpack


{'dense1': {'weight': ParamState(
    value={
      'bias': ShapedArray(float32[20]),
      'weight': ShapedArray(float32[10,20])
    }
  )},
 'dense2': {'weight': ParamState(
    value={
      'bias': ShapedArray(float32[30]),
      'weight': ShapedArray(float32[20,30])
    }
  )}}

## Demonstrate the Loaded Model
Let's run the loaded model and check if it produces the same output as the original model.

In [8]:
y1 = model1(jnp.ones((1, 10)))
y2 = model2(jnp.ones((1, 10)))
y3 = model3(jnp.ones((1, 10)))
print(jnp.allclose(y1, y2))    # True
print(jnp.allclose(y1, y3))    # True

True
True
